In [2]:
import numpy as np

from mintpy.objects import sensor
from mintpy.utils import ptime, readfile, utils as ut, writefile
from mintpy.objects.stack import timeseries
import datetime as dt
import matplotlib.pyplot as plt

In [3]:
#read asc and dsc timeseriesfiles to merge them
asc_ts_file=r'C:\Users\bvarugu\Documents\merged_asc_GNSS_corrected_timeseries_TRE_GNSS.h5'
asc_obj = timeseries(asc_ts_file);
asc_ts_atr = asc_obj.get_metadata();
asc_dates = asc_obj.get_date_list();

dsc_ts_file=r'C:\Users\bvarugu\Documents\merged_dsc_GNSS_corrected_timeseries_TRE_GNSS.h5'
dsc_obj = timeseries(dsc_ts_file);
dsc_ts_atr = dsc_obj.get_metadata();
dsc_dates = dsc_obj.get_date_list();
print('Ascending dates \n',asc_dates);
print('Descending dates \n',dsc_dates);

Ascending dates 
 ['20220319', '20220331', '20220412', '20220424', '20220506', '20220518', '20220530', '20220611', '20220623', '20220705', '20220717', '20220729', '20220810', '20220822', '20220903', '20220915', '20220927', '20221009', '20221021', '20221102', '20221114', '20221126', '20221208', '20221220', '20230101', '20230113', '20230125', '20230206', '20230218', '20230302', '20230314', '20230326', '20230407', '20230419', '20230501', '20230513', '20230525', '20230606', '20230618', '20230630', '20230712', '20230724', '20230805', '20230817', '20230829', '20230910', '20230922', '20231004', '20231016', '20231028', '20231109', '20231121', '20231203', '20231215', '20231227', '20240108', '20240120', '20240201', '20240213', '20240225', '20240308', '20240320', '20240401', '20240413', '20240425', '20240507', '20240519', '20240531', '20240612', '20240706', '20240718', '20240730', '20240811', '20240823', '20240904', '20240916', '20240928', '20241010', '20241022', '20241103']
Descending dates 
 ['

In [4]:
if asc_ts_atr['REF_DATE'] != dsc_ts_atr['REF_DATE']:
    print('***WARNING****');
    print('REFerence date not smae. Change accordingly')

In [5]:
asc_date_stamps = [dt.datetime.strptime(date, '%Y%m%d') for date in asc_dates];
dsc_date_stamps = [dt.datetime.strptime(date, '%Y%m%d') for date in dsc_dates];


In [6]:
asc_dataset1 = readfile.read(asc_ts_file, datasetName=asc_ts_atr['REF_DATE'])[0];
dsc_dataset1 = readfile.read(dsc_ts_file, datasetName=dsc_ts_atr['REF_DATE'])[0];
asc_mask = np.isnan(asc_dataset1);
dsc_mask = np.isnan(dsc_dataset1);
combined_mask = asc_mask + dsc_mask;

In [7]:
from dateutil import rrule


monthly_date_stamps = list(rrule.rrule(rrule.MONTHLY,dtstart=asc_date_stamps[0].replace(day=1),until=asc_date_stamps[-1]));
if ((asc_date_stamps[0]-monthly_date_stamps[0]).days > 15) or  ((dsc_date_stamps[0]-monthly_date_stamps[0]).days > 15):
    monthly_date_stamps = monthly_date_stamps[1:]
print(monthly_date_stamps)
monthly_date_stamps = monthly_date_stamps[:-2];
print('Removing last two months')

[datetime.datetime(2022, 4, 1, 0, 0), datetime.datetime(2022, 5, 1, 0, 0), datetime.datetime(2022, 6, 1, 0, 0), datetime.datetime(2022, 7, 1, 0, 0), datetime.datetime(2022, 8, 1, 0, 0), datetime.datetime(2022, 9, 1, 0, 0), datetime.datetime(2022, 10, 1, 0, 0), datetime.datetime(2022, 11, 1, 0, 0), datetime.datetime(2022, 12, 1, 0, 0), datetime.datetime(2023, 1, 1, 0, 0), datetime.datetime(2023, 2, 1, 0, 0), datetime.datetime(2023, 3, 1, 0, 0), datetime.datetime(2023, 4, 1, 0, 0), datetime.datetime(2023, 5, 1, 0, 0), datetime.datetime(2023, 6, 1, 0, 0), datetime.datetime(2023, 7, 1, 0, 0), datetime.datetime(2023, 8, 1, 0, 0), datetime.datetime(2023, 9, 1, 0, 0), datetime.datetime(2023, 10, 1, 0, 0), datetime.datetime(2023, 11, 1, 0, 0), datetime.datetime(2023, 12, 1, 0, 0), datetime.datetime(2024, 1, 1, 0, 0), datetime.datetime(2024, 2, 1, 0, 0), datetime.datetime(2024, 3, 1, 0, 0), datetime.datetime(2024, 4, 1, 0, 0), datetime.datetime(2024, 5, 1, 0, 0), datetime.datetime(2024, 6, 1, 0

In [8]:
asc_date_group = {monthly_date.strftime('%Y%m%d'): [] for monthly_date in monthly_date_stamps}
dsc_date_group = {monthly_date.strftime('%Y%m%d'): [] for monthly_date in monthly_date_stamps}

for monthly_date in monthly_date_stamps:
    for date in asc_date_stamps:
        if abs((date - monthly_date).days) <= 15:
            asc_date_group[monthly_date.strftime('%Y%m%d')].append(date.strftime('%Y%m%d'))
    for date in dsc_date_stamps:
        if abs((date - monthly_date).days) <= 15:
            dsc_date_group[monthly_date.strftime('%Y%m%d')].append(date.strftime('%Y%m%d'))
 ##*** fix errors related to the dates corresoding to 15th as they get recorded in the consecutive months       
            

In [9]:
monthly_date_list = [datestamp.strftime('%Y%m%d')for datestamp in monthly_date_stamps]

if (asc_ts_atr['LENGTH'] != dsc_ts_atr['LENGTH']) or (asc_ts_atr['WIDTH'] != dsc_ts_atr['WIDTH']):
    raise ValueError('Asc and Dsc datasets are not equal')


In [10]:
length = int(asc_ts_atr['LENGTH'])
width = int(asc_ts_atr['WIDTH'])
monthly_ts = np.zeros([len(monthly_date_list),length, width]) * np.nan;

In [11]:

for i, monthly_date in enumerate(monthly_date_list):
    asc_group_dates = asc_date_group[monthly_date];dsc_group_dates = dsc_date_group[monthly_date];
    images_per_month = len(asc_group_dates) + len(dsc_group_dates)
    monthly_data = np.zeros([images_per_month,length, width]) * np.nan;
    temp =0;
    for date in asc_group_dates:
        monthly_data[temp,:,:]= readfile.read(asc_ts_file, datasetName=date)[0];
        temp = temp+1;
    for date in dsc_group_dates:
        monthly_data[temp,:,:]= readfile.read(dsc_ts_file, datasetName=date)[0];
        temp = temp+1;
    
    monthly_ts[i,:,:] = np.nanmean(monthly_data,axis=0);


C:\Users\bvarugu\AppData\Local\Temp\ipykernel_30088\1934534159.py:13: RuntimeWarning: Mean of empty slice
  monthly_ts[i,:,:] = np.nanmean(monthly_data,axis=0);


In [12]:
#make first date to zeros
monthly_ts[0,:,:] *= 0 ;
monthly_ts[:, combined_mask] = np.nan

In [ ]:
atr = dict()
for key, value in asc_ts_atr.items():
    atr[key] = value
atr['FILE_TYPE'] = 'timeseries'

atr['REF_DATE'] = monthly_date_list[0];

file_outname = r'C:\Users\bvarugu\Documents\merged_UP_GNSS_corrected_timeseries_TRE_GNSS.h5'
final_dates = np.array(monthly_date_list, dtype=np.bytes_);
ts_dict = {
    "date"       : final_dates,
    "timeseries" : monthly_ts,
}
writefile.write(ts_dict,file_outname,metadata=atr);

delete exsited file: C:\Users\bvarugu\Documents\merged_UP_GNSS_corrected_timeseries_TRE_GNSS.h5
create HDF5 file: C:\Users\bvarugu\Documents\merged_UP_GNSS_corrected_timeseries_TRE_GNSS.h5 with w mode
create dataset /date       of |S8        in size of (30,)                with compression=None
create dataset /timeseries of float64    in size of (30, 7201, 9001)     with compression=None
